# Fairify Artifact
Artifact of the paper "Fairify: Fairness Verification of Neural Networks" from ICSE 2023.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [1]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [2]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 33 compute_cascadelake_r nodes ready to use


### Reserve node

In [3]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [4]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,ffd66b3f-c475-48dc-a9f1-be049b2394d7
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.149 (v4) Type: fixed MAC: bc:97:e1:78:f1:a0
Network Name,sharednet1
Created At,2025-05-17T02:59:48Z
Keypair,trovi-28d289c
Reservation Id,cb6c5e8b-beb6-488b-ae73-d1bc367fab64
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [5]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.71 port 22.


Connection successful


### Setup nvidia drivers and cuda

Download drivers

In [ ]:
my_server.execute("""
sudo apt-get --purge remove -y '*cublas*' 'cuda*' 'nvidia-*' && \
sudo apt-get autoremove -y && \
sudo apt-get autoclean && \
sudo rm -rf /usr/local/cuda* && \
sudo apt update && \
sudo apt install -y nvidia-driver-470 && \
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb && \
sudo dpkg -i cuda-keyring_1.0-1_all.deb && \
sudo apt update && \
sudo apt install -y cuda-toolkit-11-2
""")

Restart nvidia services

In [ ]:
my_server.execute("sudo modprobe -r nouveau")
my_server.execute("sudo rmmod nouveau 2>/dev/null || true")
my_server.execute("sudo update-initramfs -u")
my_server.execute("sudo modprobe nvidia")

Verifiy services

In [ ]:
my_server.execute("nvidia-smi")
my_server.execute("""
if [ -L /usr/local/bin/nvcc ]; then sudo rm /usr/local/bin/nvcc; fi && \
sudo ln -s /usr/local/cuda-11.2/bin/nvcc /usr/local/bin/nvcc
""")
my_server.execute("nvcc --version")

## Run Fairify

Now, we can finally run Fairify. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [6]:
my_server.execute("rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.109.71: b'5a919e1cbf982c07b7366e30052e92d8'
  warnings.warn(
Cloning into 'Fairify'...


<Result cmd='rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git' exited=0>

### Run dependencies and first experiment

In [7]:
my_server.execute("chmod +x Fairify/reproduce-experiment.sh")
my_server.execute("bash Fairify/reproduce-experiment.sh")

PREFIX=/home/cc/miniconda
Unpacking payload ...


entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.



Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done


entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.


installation finished.
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/cc/miniconda

  added / updated specs:
    - mamba


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    c-ares-1.34.5              |       hb9d3cd8_0         202 KB  conda-forge
    ca-certificates-2025.4.26  |       hbd8a1cb_0         149 KB  conda-forge
    conda-25.3.1               |  py313h78bf25f_1         1.1 MB  conda-forge
    expat-2.7.0                |       h5888daf_0         137 KB  conda-forge
    fmt-11.1.4                 |       h07f6e7f_1         187 KB  conda-forge
    icu-75.1                   |       he02047a_0        11.6 MB  conda-forge
    keyutils-1.6.1             |       h166bdaf_0         115 KB  conda-forge
    krb5-1.21.3                |       h659f571_0         1.3 MB  conda-forge
    libarchive-3.7.7   

...

2025-05-17 03:12:54.586458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-05-17 03:12:54.586478: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2025-05-17 03:12:54.586499: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (radhofanazizi-gmail-com-power-management): /proc/driver/nvidia/version does not exist
2025-05-17 03:12:54.586673: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


==================  STARTING MODEL DF-1.h5
###################



Processing Models: 100%|██████████| 1/1 [20:16<00:00, 1216.99s/it]


INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Expected input size: 30, Actual input size: 30
Expected input size: 30, Actual input size: 30
Verifying ...
unknown
Pruning done!
0.0 % HEURISTIC PRUNING
0.0 % TOTAL PRUNING
Expected input size: 30, Actual input size: 30
Expected input size: 30, Actual input size: 30
Verifying ...
unknown
V time:  101.69
******************
21th column
0       0.0
1       1.0
2       1.0
3       0.0
4       1.0
       ... 
4495    1.0
4496    0.0
4497    1.0
4498    1.0
4499    0.0
Name: 20, Length: 4500, dtype: float64
y_true
0       0
1       0
2       0
3       0
4       1
       ..
4495    0
4496    1
4497    0
4498    1
4499    0
Length: 4500, dtype: int64
True: 1003 | False: 3497
y_pred
0        True
1        True
2        True
3       False
4        True
        ...  
4495    False
4496     True
4497    False
4498     True
4499    False
Length: 4500, dtype: bool
True: 1648 | False: 2852
prot_attr
0       0.0
1       1.0
2       1.0
3     

<Result cmd='bash Fairify/reproduce-experiment.sh' exited=0>

Results for first experiment

In [8]:
my_server.execute("csvtool readable Fairify/src/DF/DF-1.csv | less -S")

Partition_ID Verification SAT_count UNSAT_count UNK_count h_attempt h_success B_compression S_compression ST_compression H_compression T_compression SV-time S-time HV-Time H-Time Total-Time C-check V-accurate Original-acc Pruned-acc Acc-dec C1 C2
1            unknown      0         0           1         1         0         0.0           0.0           0.0            0.0           0.0           100.365 101.02 101.69  101.96 202.99     0       0          0.6091       1.0        -          
2            unknown      0         0           2         1         0         0.0           0.0           0.0            0.0           0.0           100.399 101.02 100.729 101.0  202.02     0       0          0.6091       1.0        -          
3            unknown      0         0           3         1         0         0.0           0.0           0.0            0.0           0.0           100.347 101.06 100.59  100.86 201.92     0       0          0.6091       1.0        -          
4            unkno

<Result cmd='csvtool readable Fairify/src/DF/DF-1.csv | less -S' exited=0>

In [11]:
my_server.execute("csvtool readable Fairify/src/DF/synthetic-default-predicted-DF-metrics.csv | less -S")

Partition ID Original Accuracy  Original F1 Score   Pruned Accuracy Pruned F1 DI                 SPD                  EOD                AOD                  ERD                  CNT          TI
1            0.6091111111111112 0.33647680120709167 1.0             1.0       1.0312860436654625 0.011315487014541603 0.0086764897215717 0.008923965698996894 0.009861184718996219 [0.91062222] 0.18992487920379048
2            0.6091111111111112 0.33647680120709167 1.0             1.0       1.0312860436654625 0.011315487014541603 0.0086764897215717 0.008923965698996894 0.009861184718996219 [0.91062222] 0.18992487920379048
3            0.6091111111111112 0.33647680120709167 1.0             1.0       1.0312860436654625 0.011315487014541603 0.0086764897215717 0.008923965698996894 0.009861184718996219 [0.91062222] 0.18992487920379048
4            0.6091111111111112 0.33647680120709167 1.0             1.0       1.0312860436654625 0.011315487014541603 0.0086764897215717 0.008923965698996894 0.009861184

<Result cmd='csvtool readable Fairify/src/DF/synthetic-default-predicted-DF-metrics.csv | less -S' exited=0>